In [ ]:
import sys
sys.path.append('.')
sys.path.append('..')
sys.path.append('../..')

import numpy as np
from models.cascade_method.get_curve import split_group, interp_curves
from models.cascade_method.gauss_reg import posterior_regression
from models.cascade_method.clu_curves import clustering_main
from utils.plot_func import heatmap_plot

pick_cp = np.load('pick_results/cascade/cp_picks_yz.npy', allow_pickle=True).item()
samp_list = list(pick_cp.keys())
# sampling 5 samples
sampled_index = sorted(samp_list)[15]


picks_dict = pick_cp[sampled_index]
print(picks_dict.keys())
info_dict = pick_cp[sampled_index]

pred_hyper_dict = {
    'win_k': 5, 'bw_data': 5, 'bw_para': 50, 'valid_range': 50, 'min_len': 10, 'clu_eps': 8
}

seg_map = info_dict['seg']
gth = info_dict['gth_msfeat'][0]
curve_manual = info_dict['curve_manual']
curve_auto = split_group(seg_map)
curve_auto_interp = interp_curves(curve_auto)
curve_auto_interp_cp = curve_auto_interp.copy()
for name in curve_auto_interp_cp:
    if len(curve_auto_interp_cp[name]) < pred_hyper_dict['win_k']:
        curve_auto_interp.pop(name)
        
# * smooth the curve
infer_opt = posterior_regression(win_k=pred_hyper_dict['win_k'], bw_data=pred_hyper_dict['bw_data'], bw_para=pred_hyper_dict['bw_para'])
field_auto, slope_dict = infer_opt.est_prior(curve_auto_interp, gth.shape, valid_range=pred_hyper_dict['valid_range'])
curve_concat, labels = clustering_main(curve_auto_interp, slope_dict, eps=pred_hyper_dict['clu_eps'])
curve_dict_smooth = infer_opt.infer_posterior(curve_concat, field_auto)
curve_dict_smooth_cp = curve_dict_smooth.copy()
for name in curve_dict_smooth_cp:
    if len(curve_dict_smooth[name]) < pred_hyper_dict['min_len']:
        curve_dict_smooth.pop(name)
                        
info_dict['curve_auto'] = curve_dict_smooth
info_dict['curve_auto_raw'] = curve_auto
info_dict['field_auto'] = field_auto
info_dict['curve_cluster'] = {'curve_dict': curve_concat, 'labels': labels}
if len(curve_manual) > 0:
    field_manual, _ = infer_opt.est_prior(curve_manual, gth.shape, valid_range=pred_hyper_dict['valid_range'])
else:
    field_manual = []
info_dict['field_manual'] = field_manual
heatmap_plot(gth, curve_auto_interp, save_path='test_raw.pdf', size=(1, 5))
heatmap_plot(gth, curve_dict_smooth, save_path='test_final.pdf', size=(1, 5))
